## 1. Retrieve Slice
For initial stages, using CreateSlice.ipynb from `/teaching-materials/teaching-materials/assignments/Exploring Queues`

In [8]:
# Load Fablib and Node Information
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()
import json
import traceback
import time

slice_name = "ECE6400-Final-DDoS-Sim"
slice = fablib.get_slice(slice_name)
slice.list_nodes()

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,0508639f-d515-44c8-b922-cf4cf000b6d9
Bastion Host,bastion.fabric-testbed.net
Bastion Username,kguerns_0000197554
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b760bd3f-a4f8-4173-ab48-795233235193,Bot_0,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe34:5805,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe34:5805,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
555a9e5e-b0ff-48c6-8993-8281737d8b38,Bot_1,1,2,10,default_ubuntu_20,qcow2,eduky-w15.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fec0:aad8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fec0:aad8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
79479254-4fe6-4aa0-be52-f2970613a9a2,Bot_2,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe8c:a11c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe8c:a11c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1123df7f-53b5-486b-8668-d9da4bc3fb99,Bot_3,1,2,10,default_ubuntu_20,qcow2,eduky-w16.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe38:86cf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe38:86cf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d6b39e23-3b58-4b75-a682-7e2a81d061cd,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w14.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedc:cdce,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedc:cdce,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7de7a9a2-df45-41a6-aec6-3c66f426e79b,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w18.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe87:9542,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe87:9542,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8ff2cacd-3562-475e-8c04-55158577dc6d,User,1,2,10,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe18:9c4e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe18:9c4e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
b760bd3f-a4f8-4173-ab48-795233235193,Bot_0,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe34:5805,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe34:5805,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
555a9e5e-b0ff-48c6-8993-8281737d8b38,Bot_1,1,2,10,default_ubuntu_20,qcow2,eduky-w15.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fec0:aad8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fec0:aad8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
79479254-4fe6-4aa0-be52-f2970613a9a2,Bot_2,1,2,10,default_ubuntu_20,qcow2,eduky-w8.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe8c:a11c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe8c:a11c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1123df7f-53b5-486b-8668-d9da4bc3fb99,Bot_3,1,2,10,default_ubuntu_20,qcow2,eduky-w16.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe38:86cf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe38:86cf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d6b39e23-3b58-4b75-a682-7e2a81d061cd,Controller,1,2,10,default_ubuntu_20,qcow2,eduky-w14.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fedc:cdce,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fedc:cdce,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
7de7a9a2-df45-41a6-aec6-3c66f426e79b,Target,1,2,10,default_ubuntu_20,qcow2,eduky-w18.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe87:9542,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe87:9542,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
8ff2cacd-3562-475e-8c04-55158577dc6d,User,1,2,10,default_ubuntu_20,qcow2,eduky-w7.fabric-testbed.net,EDUKY,ubuntu,2610:1e0:1700:206:f816:3eff:fe18:9c4e,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe18:9c4e,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


### 2. Load Nodes

In [30]:
# Setup constants
nBots = 4

# Load nodes
bots = [slice.get_node(name=f"Bot_{i}") for i in range(nBots)]
target = slice.get_node(name="Target")
controller = slice.get_node(name="Controller")
user = slice.get_node(name="User")

ip_target = "10.0.0.2"

### 3. Baseline Metric Collection

In [31]:
print("\n===== Collect Baseline Metrics =====")
print("\n--- Baseline Latency & Packet Loss (Target -> Controller) ---")
print(target.execute(f"ping -i 0.5 -c 20 {controller.get_management_ip()}"))

print("\n--- Baseline Latency & Packet Loss (User -> Target) ---")
print(user.execute(f"ping -i 0.5 -c 20 {ip_target}"))

print("\n--- Baseline Queue Status on Target ---")
print(target.execute("sudo tc -s qdisc show dev enp7s0"))

#print("\n--- Baseline CPU Usage on Target ---")
#print(target.execute("top -bn1 | head -n 10"))

#print("\n--- Baseline Network Traffic Stats on Target ---")
#print(target.execute("ifconfig enp7s0"))

time.sleep(2)


===== Collect Baseline Metrics =====

--- Baseline Latency & Packet Loss (Target -> Controller) ---
PING 2610:1e0:1700:206:f816:3eff:fedc:cdce(2610:1e0:1700:206:f816:3eff:fedc:cdce) 56 data bytes
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=1 ttl=64 time=2.03 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=2 ttl=64 time=0.322 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=3 ttl=64 time=0.282 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=4 ttl=64 time=0.260 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=5 ttl=64 time=0.229 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=6 ttl=64 time=0.284 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=7 ttl=64 time=0.252 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=8 ttl=64 time=0.250 ms
64 bytes from 2610:1e0:1700:206:f816:3eff:fedc:cdce: icmp_seq=9 ttl=64 time=0.246 ms
64 bytes from 2610:1e0:1700:206:f816:3e

### 2. Sending instructions to the bots from the controller

In [33]:
# Reduce target's queue size to 1mb
target.execute("sudo tc qdisc add dev enp7s0 root tbf rate 10Mbit limit 1kb burst 10kb")

# Start DDoS simulation
print("\n===== Starting DDoS Simulation =====")
for bot in bots:
    print(f"Starting simulated attack from {bot.get_name()}...")
    bot.execute(f"sudo ping -f -c 1000 {ip_target} &") # Run ping flood in background

# Collect statistics
print("\nDDoS simulation in progress. Monitoring effect...")
time.sleep(5) # 5 seconds

print("\n===== Monitoring Effects During Attack =====")
print("\n--- Latency & Packet Loss (Target -> Controller) ---")
print(target.execute(f"ping -i 0.5 -c 20 {controller.get_management_ip()}"))

print("\n--- Latency & Packet Loss (User -> Target) ---")
print(user.execute(f"ping -i 0.5 -c 20 {ip_target}"))

print("\n--- Queue Status on Target ---")
print(target.execute("sudo tc -s qdisc show dev enp7s0"))

#print("\n--- CPU Usage on Target ---")
#print(target.execute("top -bn1 | head -n 10"))

#print("\n--- Network Traffic Stats on Target ---")
#print(target.execute("ifconfig enp7s0"))

# Stop DDoS simulation
print("\n===== Stopping Attack =====")
for bot in bots:
    bot.execute("sudo pkill ping")

print("\nRestoring normal network conditions.")
target.execute("sudo tc qdisc del dev enp7s0 root") # Remove queue limit
print("\nDDoS simulation complete. Network restored.")

    


===== Starting DDoS Simulation =====
Starting simulated attack from Bot_0...
PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.

--- 10.0.0.2 ping statistics ---
1000 packets transmitted, 1000 received, 0% packet loss, time 70ms
rtt min/avg/max/mdev = 0.020/0.058/0.250/0.020 ms, ipg/ewma 0.070/0.067 ms
Starting simulated attack from Bot_1...
PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.

--- 10.0.0.2 ping statistics ---
1000 packets transmitted, 1000 received, 0% packet loss, time 70ms
rtt min/avg/max/mdev = 0.020/0.054/0.253/0.017 ms, ipg/ewma 0.070/0.065 ms
Starting simulated attack from Bot_2...
PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.

--- 10.0.0.2 ping statistics ---
1000 packets transmitted, 1000 received, 0% packet loss, time 70ms
rtt min/avg/max/mdev = 0.020/0.058/0.120/0.017 ms, ipg/ewma 0.070/0.066 ms
Starting simulated attack from Bot_3...
PING 10.0.0.2 (10.0.0.2) 56(84) bytes of data.

--- 10.0.0.2 ping statistics ---
1000 packets transmitted, 1000 received, 0% packet l

### 3. Cleanup Resources

In [3]:
# Delete Slice
try:
    #To delete the slice change "CHECK" to "True", this is to prevent accidental slice deletion
    CHECK = True
    if (CHECK):
        slice = fablib.get_slice(slice_name)
        slice.delete()
    else:
        print("Change the Boolean to delete slice")
except Exception as e:
    print(f"Fail: {e}")